In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
import torch
from transformers import  AutoTokenizer

model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

from transformers import AutoModelForCausalLM, AutoConfig

model_name = "microsoft/phi-2"

config = AutoConfig.from_pretrained(
    model_name,
    vocab_size=len(tokenizer),
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    trust_remote_code=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
tokenizer.model_max_length, tokenizer.truncation_side, tokenizer.padding_side

(2048, 'right', 'right')

In [4]:
model_name = "microsoft/phi-2"
# phi2_model_pretrained = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,  
# )
phi2_model_pretrained = AutoModelForCausalLM.from_config(config, 
                                                         trust_remote_code=True, 
                                                        #  torch_dtype=torch.float16, 
                                                         )

### Create dataset

In [5]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd 
import json
import os 
import h5py

In [6]:
def get_image_name(image_id_from_caption, list_image_info): 
    for img in list_image_info: 
        if img['id'] == image_id_from_caption: 
            img_name = img['file_name'].split('.')[0]
            return img['file_name'].split('.')[0]
    return 'NoImgNameFound'

In [7]:
# file_path_captions_coco = '/media/App/amaranth/lavanya/Capstone_data/annotations_trainval2017/annotations/captions_train2017.json'

# with open(file_path_captions_coco) as f:
#    data = json.load(f)

# captions_info = []
# for a in data['annotations']: 
#     captions_info.append([a['image_id'], a['caption'], a['id']])

# captions_info_df = pd.DataFrame(data=captions_info, columns=['image_id', 'caption', 'caption_id'])
# captions_info_df['image_name'] = captions_info_df['image_id'].apply(lambda x: get_image_name(x, data['images']))
# captions_info_df['image_name'] = captions_info_df['image_name'].apply(lambda x: '0'*(12-len(str(x))) + str(x))
# captions_info_df.to_csv('captions_images_map_COCO_train2017.csv', index = False)

In [8]:
captions_info_df = pd.read_csv('captions_images_map_COCO_train2017.csv')

/tmp/ipykernel_262814/1237926302.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  captions_info_df = pd.read_csv('captions_images_map_COCO_train2017.csv')


In [9]:
import h5py    
import numpy as np    

In [10]:
class COCO_CLIP_Dataset(Dataset):

    def __init__(
        self, caption_file, embedding_path, tokenizer, max_token_len_data):
        self.embedding_path = embedding_path
        self.caption_file = caption_file
        self.tokenizer = tokenizer
        self.max_token_len_data = max_token_len_data

    def __len__(self):
        return len(self.caption_file)
    
    def __getitem__(self, index):
        row = self.caption_file.iloc[[index]]
        df_img = row['image_id'].values[0]
        img_base_name = '0'*(12-len(str(df_img))) + str(df_img)
        img_base_name = img_base_name.replace(' ', '0')
        img_clip_embedding_path = os.path.join(self.embedding_path, f'{img_base_name}.h5')

        np_array_embed_img = h5py.File(img_clip_embedding_path,'r+')['image_features'][()]
        
        img_caption = row['caption'].values[0] ## Tokenize this 
        img_caption_tokenized = self.tokenizer(img_caption, return_tensors="pt", 
                                               return_attention_mask=False).input_ids

        ## put bos, eos, and padding for batch 
        # input_bos = torch.cat((torch.tensor(self.tokenizer.bos_token_id).view((1,1)), 
        #                                                img_caption_tokenized), dim=1)

        input_bos = img_caption_tokenized

        input_eos = torch.cat((input_bos, 
                               torch.tensor(self.tokenizer.eos_token_id).view((1,1))), dim=1)
        
        if (self.max_token_len_data - input_eos.shape[1]) > 0: 
            input_final =  torch.cat((input_eos,torch.tensor([self.tokenizer.pad_token_id]*(self.max_token_len_data - input_eos.shape[1])).unsqueeze(0)), dim=1)
        else: 
            input_final = input_eos
        
        return torch.tensor(np_array_embed_img).squeeze(0), input_final.squeeze(0)

In [11]:
def file_exists(image_id, fpath = '/media/App/amaranth/lavanya/Capstone_data/clip_features_base_patch32/'): 

    n = '0'*(12-len(str(image_id))) + str(image_id) + '.h5'
    fp = os.path.join(fpath, n)

    if os.path.exists(fp): 
        return True
    else: 
        return False

In [12]:
### captions_info_df contains for 1 image multiple entries, lets reduce keeping one image, one entry. 
captions_info_df_subset = captions_info_df.drop_duplicates(subset='image_id', keep='first')
captions_info_df_subset['image_embed_exists'] = captions_info_df_subset['image_id'].apply(lambda x: file_exists(x))

/tmp/ipykernel_262814/1249603411.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  captions_info_df_subset['image_embed_exists'] = captions_info_df_subset['image_id'].apply(lambda x: file_exists(x))


In [13]:
len(captions_info_df_subset), len(captions_info_df_subset[captions_info_df_subset['image_embed_exists'] == True])

(118287, 118287)

In [14]:
max_token_len_data = 75
dataset = COCO_CLIP_Dataset(captions_info_df_subset, 
                            '/media/App/amaranth/lavanya/Capstone_data/clip_features_base_patch32/', 
                            tokenizer, max_token_len_data)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleResBlock(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.pre_norm = nn.LayerNorm(input_size)
        self.proj = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.GELU(),
            nn.Linear(input_size, input_size)
        )
    def forward(self, x):
        x = self.pre_norm(x)
        return x + self.proj(x)
    
class Phi2wrapper(nn.Module):
    
    #This defines the structure of the NN.
    def __init__(self, input_dim_CLIP=768, input_dim_phi2=2560, 
                 phi2_model=phi2_model_pretrained, 
                 max_token_len_data=max_token_len_data, tokenizer=tokenizer):
        
        super(Phi2wrapper, self).__init__()

        self.input_dim_CLIP = input_dim_CLIP
        self.input_dim_phi2 = input_dim_phi2
        self.projection_img = nn.Linear(self.input_dim_CLIP, self.input_dim_phi2, 
                                        bias=False)
        self.resblock = SimpleResBlock(self.input_dim_phi2)
        self.phi2_model = phi2_model
        self.max_token_len_data = max_token_len_data
        self.tokenizer = tokenizer

    def forward(self, x):

        x = self.projection_img(x)
        x = self.resblock(x)

        # x = self.phi2_model.forward(inputs_embeds=x)
        x = self.phi2_model.generate(inputs_embeds=x, 
                                     max_new_tokens=self.max_token_len_data, 
                                     output_scores=True, return_dict_in_generate = True, 
                                     pad_token_id=self.tokenizer.eos_token_id)

        # x = self.phi2_model.model.layers[0](x)
        # for layer_idx in range(1, 32): 
        #     x = self.phi2_model.model.layers[layer_idx](x[0])
                
        # x = self.phi2_model.model.final_layernorm(x[0])
        # x = self.phi2_model.lm_head(x)
        
        return x 

device = 'cuda:1'
torch.set_grad_enabled(True)  
phi2_projection_model = Phi2wrapper().to(device=device)

## Freezing phi-2 for projection layer training 
for name, param in phi2_projection_model.named_parameters():
    if "phi2_model" in name:
        param.requires_grad = False
    else: 
        param.requires_grad = True

In [16]:
batch_size_train = 60 
train_dataloader = DataLoader(dataset, batch_size=batch_size_train, shuffle=True)
num_batches_train_on = 1500    
num_batches_train_on, len(train_dataloader)

(1500, 1972)

In [17]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, 
                                    phi2_projection_model.parameters()), 
                            lr=1e-5, eps=1e-9) 

In [18]:
num_epochs = 10
vocab_size = 50295

phi2_projection_model.train()
N_batches = len(train_dataloader)
                
for epoch in range(num_epochs):

    print(f"Working on epoch {epoch}")

    for iteration, batch in enumerate(train_dataloader):

        if iteration == num_batches_train_on: 
            break 

        print(f"Iteration {iteration}/{num_batches_train_on}", end='\r')

        optimizer.zero_grad()

        input = batch[0]
        gt = batch[1] 

        output = phi2_projection_model(input.to(device))

        ## need to map gt token_ids to one-hot enocding vocab_size
        gt_one_hot = torch.nn.functional.one_hot(gt, vocab_size).to(torch.float32)

        ## output in correct shape
        output_tensor_new = torch.empty(batch_size_train, max_token_len_data, vocab_size)
        for idx, s in enumerate(output.scores): 
            output_tensor_new[:, idx, :] = s
        
        output_tensor_new = F.softmax(output_tensor_new, dim=-1)
        
        ## ce loss between output_tensor_new and gt_one_hot
        loss = F.cross_entropy(output_tensor_new.view(-1, vocab_size), gt.view(-1))

        loss.requires_grad = True
        loss.backward()

        optimizer.step() 
        # optimizer.zero_grad(set_to_none=True) 

        if (iteration % 100) == 0: 
            print("")

            ## print gt and output decoded tokens for visual inspection for the 1st el of batch
            gt_input_ids = gt[0]
            output_input_ids = torch.argmax(output_tensor_new[0], dim=1)
            output_sequences_input_ids = output.sequences[0, :]

            gt_idx_0_decoded = tokenizer.decode(gt_input_ids).replace('<|endoftext|>', '')
            output_idx_0_decoded = tokenizer.decode(output_input_ids).replace('<|endoftext|>', '')
            output_sequences_idx_0_decoded = tokenizer.decode(output_sequences_input_ids).replace('<|endoftext|>', '')

            # print(f"Loss: {loss}\nInput_ids (gt): {gt_input_ids}\nInput_ids (pred): {output_input_ids}\nInput_ids (sequence): {output_sequences_input_ids}")
            ## print loss 
            print(f"Loss: {loss}\nCaption (gt): {gt_idx_0_decoded}\nCaption (pred): {output_idx_0_decoded}\nCaption (sequence): {output_sequences_idx_0_decoded}")
 
        
    print("")
    print(f"Epoch {epoch} finished")
    print("")

Working on epoch 0
Iteration 0/1500
Loss: 10.825667381286621
Caption (gt): A close up of a grill loaded with burgers and hot dogs.
Caption (pred):  arrives2011 urgently presently Productions Atkins angered Myrirling Productionsة Carm SwedASON reviewersTG objective Carm MyrvertASON Renew Permanentolescent titaniumolescent Renew SwedTG rushScientacking83 helpedborgTG objectiveectiveо borrowersolescentة rebirthTG ensures cigarettes totally libraries glorybec stripedholeolescent pianoricalirling rampage Faction investigated reviewers DepartmentTGolescentollah Renew presently Productions piracy unresウ▄ Wealth408 Renoicultural
Caption (sequence):  arrives2011 urgently presently Productions Atkins angered Myrirling Productionsة Carm SwedASON reviewersTG objective Carm MyrvertASON Renew Permanentolescent titaniumolescent Renew SwedTG rushScientacking83 helpedborgTG objectiveectiveо borrowersolescentة rebirthTG ensures cigarettes totally libraries glorybec stripedholeolescent pianoricalirling r